In [1]:
import pickle
import numpy as np
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

In [2]:
lstm_result_path = "data/LSTM_enhanced_results.pickle"
classifiers_result_path = "data/classifiers_results.pickle"

In [3]:
def compare(first, second):
        second = pd.DataFrame(second)
        second.columns = [i+"_y" for i in second.columns]
        res = pd.concat([first,second],axis=1)
  
        res[['scores_x', 'scores_y']] = res[['scores_x', 'scores_y']].round(decimals=0)

        yes_yes = res[(res['actual_x'] != res['scores_x']) & (res['actual_x'] != res['scores_y'])].shape[0]
        yes_no = res[(res['actual_x'] != res['scores_x']) & (res['actual_x'] == res['scores_y'])].shape[0]
        no_yes = res[(res['actual_x'] == res['scores_x']) & (res['actual_x'] != res['scores_y'])].shape[0]
        no_no = res[(res['actual_x'] == res['scores_x']) & (res['actual_x'] == res['scores_y'])].shape[0]

        table = np.array([[yes_yes, yes_no], [no_yes, no_no]])
        test = mcnemar(table, exact=False)
        dictionary = dict(pvalue=test.pvalue, statistic=test.statistic)
        return dictionary

In [6]:
with open(lstm_result_path, "rb") as fp:
    lstm_dict = pickle.load(fp)
with open(classifiers_result_path, "rb") as fp:
    classifiers_dict = pickle.load(fp)
result = {}

lstm_dict = pd.DataFrame(lstm_dict)
lstm_dict.columns = [i+"_x" for i in lstm_dict.columns]

for i in classifiers_dict.keys():
    result['LSTM vc ' + str(i)] = compare(lstm_dict, classifiers_dict[i])

with open("../pickle/McNemar.pickle", "wb") as fp:
    pickle.dump(result, fp)

for i in result.keys():   
    print(i,result[i])

LSTM vc Gradient Boosting {'pvalue': 3.624358498759576e-05, 'statistic': 17.05861456483126}
LSTM vc Bagging NN {'pvalue': 8.019335217437568e-11, 'statistic': 42.25308641975309}
LSTM vc SVM {'pvalue': 2.7759987542465218e-18, 'statistic': 76.04267701260912}
LSTM vc Random Forest {'pvalue': 1.024707295783294e-47, 'statistic': 210.58333333333334}
LSTM vc Logistic Regression {'pvalue': 1.150384773651047e-29, 'statistic': 127.95117428924598}
